# SAXPY Example

$$ y = ax + y $$
위 연산에 대하여 벡터 계산

## CPU Version

In [1]:
%%file saxpy_cpu.cc

#include <stdio.h>
#include <stdlib.h>
#include "utils.h"

// saxpy: c = alpha * a + b
void saxpy(float* a, float* b, float *c, float alpha, int n_size) {
    for (int i = 0; i < n_size; i++) {
        c[i] = alpha * a[i] + b[i];
    }
}

int main() {
    float *a, *b, *c;
    int n_size = 65536;
    
    init_rand();
    
    a = init_vector(n_size);
    b = init_vector(n_size);
    c = init_vector(n_size);
    
    printf("[A]\n");
    print_vector(a, 100);
    printf("[B]\n");
    print_vector(b, 100);
    printf("[C]\n");
    print_vector(c, 100);
    
    saxpy(a, b, c, 2.0, n_size);
    
    printf("[saxpy:: c = alpha * a + b]\n");
    print_vector(c, 100);
    
    free(a);
    free(b);
    free(c);
    
    return 0;
}


Overwriting saxpy_cpu.cc


### Compile 및 실행

In [2]:
! gcc -Wall -o saxpy_cpu saxpy_cpu.cc utils.cc && ./saxpy_cpu

[A]
0.22 0.07 0.03 0.89 0.01 0.23 0.85 0.99 0.37 0.26 
0.06 0.25 0.51 0.85 0.66 0.12 0.80 0.87 0.94 0.11 
0.69 0.42 0.63 1.00 0.93 0.55 0.24 0.62 0.76 0.20 
0.46 0.98 0.28 0.49 0.88 0.29 0.72 0.73 0.28 0.09 
0.98 0.34 0.34 0.49 0.19 0.00 0.61 0.99 0.87 0.56 
0.10 0.57 0.98 0.73 0.56 0.91 0.28 0.80 0.53 0.04 
0.01 0.99 0.02 0.28 0.48 0.89 0.57 0.20 0.62 0.86 
0.29 0.61 0.19 0.64 0.10 0.38 0.64 0.71 0.37 0.51 
0.27 0.47 0.08 0.25 0.20 0.64 0.16 0.48 0.44 0.68 
0.52 0.45 0.67 0.53 0.73 0.15 0.43 0.30 0.35 0.05 
[B]
0.96 0.25 0.86 0.49 0.46 0.08 0.18 0.22 0.06 0.37 
0.82 0.55 0.28 0.51 0.31 0.07 0.19 0.13 0.47 0.27 
0.72 0.31 0.72 0.95 0.51 0.47 0.25 0.53 0.45 0.95 
0.28 0.40 0.19 0.14 0.89 0.66 0.22 0.07 0.88 0.28 
0.44 0.70 0.83 0.72 0.21 0.14 0.79 0.40 0.27 0.26 
0.67 0.99 0.57 0.39 0.94 0.08 0.85 0.19 0.62 0.30 
0.13 0.89 0.70 0.33 0.03 0.60 0.98 0.25 0.67 0.86 
0.53 0.11 0.56 0.36 0.83 0.76 0.50 0.62 0.16 0.76 
0.88 0.83 0.75 0.45 0.22 0.69 0.54 0.07 0.87 0.15 
0.36 0.01 0.04 0.07 0.3

## GPU Version

In [3]:
%%file saxpy_gpu.cu

#include "utils.h"

// CUDA Kernel function
__global__ 
void d_saxpy(float* a, float* b, float* c, float alpha, int n_size) {
    int i = blockDim.x * blockIdx.x + threadIdx.x;
    c[i] = alpha * a[i] + b[i];
}

// computation
void saxpy(float* a, float* b, float* c, float alpha, int n_size) {
    for (int i = 0; i < n_size; i++) {
        c[i] = alpha * a[i] + b[i];
    }
}

int main() {
    float *a, *b, *c, *cdef;
    int n_size = 65536;
    int bufsize= n_size * sizeof(float);
    
    a = init_vector(n_size);
    b = init_vector(n_size);
    c = init_vector(n_size);
    cdef = init_vector(n_size);
    
    // Step 1. Create GPU memory
    float *d_a, *d_b, *d_c;
    cudaMalloc((void**)&d_a, bufsize);
    cudaMalloc((void**)&d_b, bufsize);
    cudaMalloc((void**)&d_c, bufsize);
    
    // Step 2. Copy to GPU memory
    cudaMemcpy(d_a, a, bufsize, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, b, bufsize, cudaMemcpyHostToDevice);
    cudaMemset(d_c, 0.f, bufsize);
    
    // Step 3. Kernel Call
    saxpy(a, b, cdef, 2.0, n_size);
    
    dim3 blockDim(16);
    dim3 gridDim((n_size + blockDim.x - 1) / blockDim.x);
    d_saxpy<<< gridDim, blockDim >>>(d_a, d_b, d_c, 2.0, n_size);

    // Step 4. Copy from GPU
    cudaMemcpy(c, d_c, n_size * sizeof(float), cudaMemcpyDeviceToHost);

    // Step 5. Check Result
    check_result(c, cdef, n_size);
    
    // Step 6. Finalize GPU memory
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
    
    free(a);
    free(b);
    free(c);
    free(cdef);
    
    return 0;
}


Overwriting saxpy_gpu.cu


### Compile 및 실행

In [4]:
! nvcc -arch=sm_30 -o saxpy_gpu saxpy_gpu.cu utils.cc && ./saxpy_gpu

Complete!!

## __host__ __device__의 활용
GPU와 CPU간에 동일한 코드를 사용하게 된다면 \__host\__ \__device\__ 를 이용하는 것을 고려해 볼 수 있습니다. 이것을 이용하면 nvcc 컴파일러는 CPU와 GPU에서 모두 사용하는 코드라고 인식하여, CPU와 GPU에서 모두 사용할 수 있는 함수를 만들 수 있습니다.

위 예제를 보면 SAXPY라는 동작 자체는 CPU와 GPU 모두 동일한 연산을 하고 있습니다. 따라서 하나의 함수로 통합할 수 있다면 CPU에서 검증된 알고리즘 코드를 GPU에서 쉽게 동작시켜 볼 수 있게 되고, 코드를 관리하는데도 도움이 됩니다.

In [5]:
%%file saxpy_host_device.cu

#include "utils.h"

__host__ __device__
float _saxpy(float alpha, float a, float b) {
    return alpha * a + b;
}

// CUDA Kernel function
__global__ 
void d_saxpy(float* a, float* b, float* c, float alpha, int n_size) {
    int i = blockDim.x * blockIdx.x + threadIdx.x;
    c[i] = _saxpy(alpha, a[i], b[i]);
}

// computation
void saxpy(float* a, float* b, float* c, float alpha, int n_size) {
    for (int i = 0; i < n_size; i++) {
        c[i] = _saxpy(alpha, a[i], b[i]);
    }
}

int main() {
    float *a, *b, *c, *cdef;
    int n_size = 65536;
    int bufsize= n_size * sizeof(float);
    
    a = init_vector(n_size);
    b = init_vector(n_size);
    c = init_vector(n_size);
    cdef = init_vector(n_size);
    
    // Step 1. Create GPU memory
    float *d_a, *d_b, *d_c;
    cudaMalloc((void**)&d_a, bufsize);
    cudaMalloc((void**)&d_b, bufsize);
    cudaMalloc((void**)&d_c, bufsize);
    
    // Step 2. Copy to GPU memory
    cudaMemcpy(d_a, a, bufsize, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, b, bufsize, cudaMemcpyHostToDevice);
    cudaMemset(d_c, 0.f, bufsize);
    
    // Step 3. Kernel Call
    saxpy(a, b, cdef, 2.0, n_size);
    
    dim3 blockDim(16);
    dim3 gridDim((n_size + blockDim.x - 1) / blockDim.x);
    d_saxpy<<< gridDim, blockDim >>>(d_a, d_b, d_c, 2.0, n_size);

    // Step 4. Copy from GPU
    cudaMemcpy(c, d_c, n_size * sizeof(float), cudaMemcpyDeviceToHost);

    // Step 5. Check Result
    check_result(c, cdef, n_size);
    
    // Step 6. Finalize GPU memory
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
    
    free(a);
    free(b);
    free(c);
    free(cdef);
    
    return 0;
}


Overwriting saxpy_host_device.cu


In [6]:
! nvcc -arch=sm_30 -o saxpy_host_device saxpy_host_device.cu utils.cc && ./saxpy_host_device

Complete!!

## Template의 활용
CUDA 컴파일러는 C/C++을 지원하므로 C++의 다양한 기능들을 이용할 수 있습니다. 그 중 하나는 template이 있는데요, 자료형에 따라서 커널함수를 생성할 필요가 없어진다는 장점이 있습니다. 물론 다양한 자료형을 지원하게 되는 경우 이것은 더이상 SAXPY가 아니라 DAXPY와 같이 다른 연산이 되어버리지만 개념적으로 설명을 위해 예를 들어보겠습니다.

진정한 C++ 코드로 넘어가기 위해선 손대야 할 곳이 많은데요, 아래 예제 코드에서는 Kernel 함수만 바꿔서 보여드리겠습니다.

In [7]:
%%file saxpy_host_device_template.cu

#include "utils.h"

template <typename T>
__host__ __device__
T _saxpy(T alpha, T a, T b) {
    return alpha * a + b;
}

// CUDA Kernel function
template <typename T>
__global__ 
void d_saxpy(T* a, T* b, T* c, T alpha, int n_size) {
    int i = blockDim.x * blockIdx.x + threadIdx.x;
    c[i] = _saxpy(alpha, a[i], b[i]);
}

// computation
template <typename T>
void saxpy(T* a, T* b, T* c, T alpha, int n_size) {
    for (int i = 0; i < n_size; i++) {
        c[i] = _saxpy(alpha, a[i], b[i]);
    }
}

int main() {
    float *a, *b, *c, *cdef;
    int n_size = 65536;
    int bufsize= n_size * sizeof(float);
    
    a = init_vector(n_size);
    b = init_vector(n_size);
    c = init_vector(n_size);
    cdef = init_vector(n_size);
    
    // Step 1. Create GPU memory
    float *d_a, *d_b, *d_c;
    cudaMalloc((void**)&d_a, bufsize);
    cudaMalloc((void**)&d_b, bufsize);
    cudaMalloc((void**)&d_c, bufsize);
    
    // Step 2. Copy to GPU memory
    cudaMemcpy(d_a, a, bufsize, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, b, bufsize, cudaMemcpyHostToDevice);
    cudaMemset(d_c, 0.f, bufsize);
    
    // Step 3. Kernel Call
    saxpy(a, b, cdef, 2.f, n_size);
    
    dim3 blockDim(16);
    dim3 gridDim((n_size + blockDim.x - 1) / blockDim.x);
    d_saxpy<<< gridDim, blockDim >>>(d_a, d_b, d_c, 2.f, n_size);

    // Step 4. Copy from GPU
    cudaMemcpy(c, d_c, n_size * sizeof(float), cudaMemcpyDeviceToHost);

    // Step 5. Check Result
    check_result(c, cdef, n_size);
    
    // Step 6. Finalize GPU memory
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
    
    free(a);
    free(b);
    free(c);
    free(cdef);
    
    return 0;
}


Overwriting saxpy_host_device_template.cu


In [8]:
! nvcc -arch=sm_30 -o saxpy_host_device_template saxpy_host_device_template.cu utils.cc && ./saxpy_host_device_template

Complete!!

### utils
다음은 위 예제 코드를 간단하게(?) 보이게 만들기 위해서 공통적으로 사용하는 기능들을 모아놓은 코드들입니다.

In [9]:
%%file utils.h
#ifndef _UTILS_H_
#define _UTILS_H_

void init_rand();
float* init_vector(int n_size);
void print_vector(float* p_vector, int n_size);
void check_result(float* a, float* b, int n_size);

#endif // _UTILS_H_

Overwriting utils.h


In [10]:
%%file utils.cc
#include <time.h>
#include <stdio.h>
#include <stdlib.h>
#include "utils.h"

void init_rand() {
    srand (time(0));
}

float* init_vector(int n_size) {
    // buffer create
    float* p_vector = (float*)malloc(n_size * sizeof(float));
    
    // initialize vector
    for (int i = 0; i < n_size; i++) {
        p_vector[i] = (float)rand() / (float)(RAND_MAX);
    }
    
    return p_vector;
}

void print_vector(float* p_vector, int n_size) {
    for (int j = 0; j < n_size / 10; j++) {
        for (int i = 0; i < 10; i++) {
//            cout << setw(5) << fixed << setprecision( 2 ) << p_vector[10*j + i];
            printf("%3.2f ", (float)p_vector[10*j + i]);
        }
        printf("\n");
    }
}

void check_result(float* a, float* b, int n_size) {
    int diff_count = 0;
    for (int i = 0; i < n_size; i++) {
        diff_count += a[i] != b[i] ? 1 : 0;
    }
    
    if (diff_count == 0)
        printf("Complete!!");
    else
        printf("Values are mis-matching from CPU & GPU");
}

Overwriting utils.cc
